# Generating a weighted Monte Carlo lightcone of dark matter halos

This notebook demonstrates how to generate a lightcone of dark matter halos with mass assembly histories.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from jax import random as jran
ran_key = jran.key(0)

### Generate a lightcone of dark matter halos

Use the `mc_weighted_halo_lightcone` function to generate a population of central halos.

In [ ]:
from diffsky.experimental.mc_lightcone_halos import mc_weighted_halo_lightcone

num_halos = 5_000
z_min, z_max = 0.01, 2.0
lgmp_min, lgmp_max = 10.5, 15.5
sky_area_degsq = 1.0

ran_key, lc_key = jran.split(ran_key, 2)
args =(
    lc_key,
    num_halos,
    z_min,
    z_max,
    lgmp_min,
    lgmp_max,
    sky_area_degsq)

cenpop = mc_weighted_halo_lightcone(*args)
print(cenpop.keys())
print(np.mean(np.isnan(cenpop['mah_params'].logm0)))

#### Sanity check the `logmp_obs` column agrees when recomputed

In [ ]:
from diffmah import mah_halopop
t0 = 13.8
tarr = np.linspace(0.1, t0, 200)

dmhdt, log_mah = mah_halopop(cenpop['mah_params'], tarr, np.log10(t0))

from jax import vmap
from jax import jit as jjit
from jax import numpy as jnp
interp_vmap = jjit(vmap(jnp.interp, in_axes=(0, None, 0)))

logmp_obs = interp_vmap(cenpop["t_obs"], tarr, log_mah)
assert np.allclose(logmp_obs, cenpop['logmp_obs'], rtol=1e-3)

#### Examine the range of `{m_obs, z_obs}` spanned by the population

Halo mass and redshift uniformly span the input ranges

In [ ]:
fig, ax = plt.subplots(1, 1)
yscale = ax.set_yscale('log')
__=ax.scatter(cenpop['z_obs'], 10**cenpop['logmp_obs'], s=1)
xlabel = ax.set_xlabel(r'$z_{\rm obs}$')
ylabel = ax.set_ylabel(r'$M_{\rm halo}\ [M_{\odot}]$')

#### For the _weighted_ lightcone, each halo has multiplicity according to its abundance in the volume

This `nhalos` column needs to be taken into account when predicting summary statistics from the weighted lightcone.

In [ ]:
fig, ax = plt.subplots(1, 1)
__=ax.loglog()
__=ax.scatter(10**cenpop['logmp_obs'], cenpop['nhalos'], s=1)
xlabel = ax.set_xlabel(r'$M_{\rm halo}\ [M_{\odot}]$')
ylabel = ax.set_ylabel(r'$N_{\rm halos}$')

#### Calculate the halo mass function, accounting for halo weights

The unweighted version uniformly spans $\log_{10}M_{\rm halo}$. The weighted version has the expected Schechter-type shape of the HMF.

In [ ]:
fig, ax = plt.subplots(1, 1)
yscale = ax.set_yscale('log')
__=ax.hist(cenpop['logmp_obs'], bins=100, alpha=0.7, label=r'${\rm unweighted}$')
__=ax.hist(cenpop['logmp_obs'], bins=100, weights=cenpop['nhalos'], alpha=0.7, label=r'${\rm weighted}$')
xlabel = ax.set_xlabel(r'$\log_{10}M_{\rm halo}/M_{\odot}$')
ylabel = ax.set_ylabel(r'$N_{\rm halos}$')
leg = ax.legend()

#### Calculate $n_{\rm halo}(z),$ accounting for halo weights

The unweighted version uniformly spans $z_{\rm min}<z<z_{\rm max}$. The weighted version accounts for the redshift evolution of the cosmological volume element and the mass function.

In [ ]:
fig, ax = plt.subplots(1, 1)
yscale = ax.set_yscale('log')
__=ax.hist(cenpop['z_obs'], bins=100, alpha=0.7, label=r'${\rm unweighted}$')
__=ax.hist(cenpop['z_obs'], bins=100, weights=cenpop['nhalos'], alpha=0.7, label=r'${\rm weighted}$')
xlabel = ax.set_xlabel(r'${\rm redshift}$')
ylabel = ax.set_ylabel(r'$N_{\rm halos}$')
leg = ax.legend()

### Examine the diversity of mass assembly histories of the halos

Each generated halo has diffmah parameters that let us compute its mass assembly history with the `mah_halopop` function. This function has a required argument `lgt0` specifying the base-10 log of the age of the universe at $z=0.$ In general, this should be set consistently with the assumed cosmology. For purposes of this demo, we will just use $t_0=13.8$ Gyr.

In [ ]:
from diffmah import mah_halopop

t0 = 13.8
lgt0 = np.log10(t0)
tarr = np.linspace(0.5, t0, 100)
dhmdt, log_mah = mah_halopop(cenpop['mah_params'], tarr, lgt0)

In [ ]:
fig, ax = plt.subplots(1, 1)
yscale = ax.set_yscale('log')

n_plot = 10
for i in range(n_plot):
    __=ax.plot(tarr, 10**log_mah[i, :])

xlabel = ax.set_xlabel(r'${\rm cosmic\ time\ [Gyr]}$')
ylabel = ax.set_ylabel(r'$M_{\rm halo}\ [M_{\odot}]$')